# Log文件解析脚本
 
目前，已经可以从Log中解析以下表格中的内容：  

|类型|正则名称|数据名称|  
|:-|:-|:-|
|MCLoc  |mcloc_re|t, x, y, theta, confidence|  
|Yaw|yaw_re|yaw_t, yaw_data, yaw_time, ax, ay, az, gx, gy, gz, offx, offy, offz|  
|Odometer|odometer_re|odo_t, odo_time, odo_x, odo_y, odo_theta|  
|下发速度|vel_re|vx, vy, omega|  
|TEB下发速度|send_vel_re|send_vel_t, send_vx, send_vy, send_vw, send_steer_angle|  
|TEB获取速度|get_vel_re|get_vel_t, get_vel_x, get_vel_vw, get_vel_steer_angle|  

##  如何添加新的解析内容
1. [正则模块中添加新的正则表达式和对应的变量](#No1)  
2. [解析函数中处理解析出来的数据，赋予对应变量](#No2)  
3. [运行解析函数，并且画图](#No3)

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import math

## <span id="No1">正则 </span>

* 定位

In [2]:
mcloc_re = re.compile("\[.*? ([0-9][0-9]):([0-9][0-9]):(\d+\.\d+)\].*x:(.*?):y:(.*?):theta:(.*?):confidence:(.*?)$")
t = []
x = []
y = []
theta =[]
confidence = []

 * 陀螺仪

In [3]:
yaw_re = re.compile("\[.*? ([0-9][0-9]):([0-9][0-9]):(\d*\.?\d*)\].*yaw: (.*?),? ?timestamp: (\d+)")
yaw_t = []
yaw_data = []
yaw_time = []
acc_re = re.compile("acc = \[(.*?), (.*?), (.*?)\]w = \[(.*?), (.*?), (.*?)\];off = \[(.*?), (.*?), (.*?)\]")
ax = []
ay = []
az = []
gx = []
gy = []
gz = []
offx = []
offy = []
offz = []

* 里程

In [4]:
odometer_re = re.compile("\[.*? ([0-9][0-9]):([0-9][0-9]):(\d*\.?\d*)\].*timestamp: (\d+), x: (.*?), y: (.*?), angle: (.*?),")
odo_t = []
odo_time = []
odo_x = []
odo_y = []
odo_theta = []

* 下发速度

In [5]:
vel_re = re.compile('\[.*? ([0-9][0-9]):([0-9][0-9]):(\d*\.?\d*)\].*{ "vx" : (.*?), "vy" : (.*?), "w" : (.*?) }')
vel_t = []
vx =[]
vy = []
omega = []

* TEB中下发速度

In [6]:
send_vel_re = re.compile('\[.*? ([0-9][0-9]):([0-9][0-9]):(\d*\.?\d*)\].* vx = (.*?); vy = (.*?); vw = (.*?); steer_angle = (.*?); max_vel = .*')
send_vel_t = []
send_vx = []
send_vy = []
send_vw = []
send_steer_angle = []

* TEB获取速度

In [7]:
get_vel_re = re.compile('\[.*? ([0-9][0-9]):([0-9][0-9]):(\d*\.?\d*)\].* vx = (.*?); omega = (.*?); steer_angle = (.*?)\n')
get_vel_t = []
get_vx = []
get_vw = []
get_steer_angle = []

## <span id="No2">解析函数 </span>

In [8]:
def read_log(filename): 
    for line in open(filename): 
        m = mcloc_re.match(line)
        yaw = yaw_re.match(line)
        odo = odometer_re.match(line)
        acc = acc_re.match(line)
        vel = vel_re.match(line)
        send_vel = send_vel_re.match(line)
        get_vel = get_vel_re.match(line)
        if m:
            t.append(float(m.group(1))*60.0*60.0+float(m.group(2))*60.0+float(m.group(3)))
            x.append(float(m.group(4)))
            y.append(float(m.group(5)))
            theta.append(float(m.group(6)))
            confidence.append(float(m.group(7)))
        if yaw:
            yaw_t.append(float(yaw.group(1))*60.0*60.0+float(yaw.group(2))*60.0+float(yaw.group(3)))
            yaw_data.append((float(yaw.group(4))/math.pi *  180.0))
            yaw_time.append(float(yaw.group(5)))
        if acc:
            ax.append(float(acc.group(1)))
            ay.append(float(acc.group(2)))
            az.append(float(acc.group(3)))
            gx.append(float(acc.group(4))/math.pi*180.0*16.4)
            gy.append(float(acc.group(5))/math.pi*180.0*16.4)
            gz.append(float(acc.group(6))/math.pi*180.0*16.4)
            offx.append(float(acc.group(7)))
            offy.append(float(acc.group(8)))
            offz.append(float(acc.group(9)))
        if odo:
            odo_t.append(float(odo.group(1))*60.0*60.0+float(odo.group(2))*60.0+float(odo.group(3)))
            odo_time.append(float(odo.group(4)))
            odo_x.append(float(odo.group(5)))
            odo_y.append(float(odo.group(6)))
            odo_theta.append((float(odo.group(7))/math.pi * 180.0))
        if vel:
            vel_t.append(float(vel.group(1))*60.0*60.0+float(vel.group(2))*60.0+float(vel.group(3)))
            vx.append(float(vel.group(4)))
            vy.append(float(vel.group(5)))
            omega.append(float(vel.group(6)))
        if send_vel:
            send_vel_t.append(float(send_vel.group(1))*60.0*60.0+float(send_vel.group(2))*60.0+float(send_vel.group(3)))
            send_vx.append(float(send_vel.group(4)))
            send_vy.append(float(send_vel.group(5)))
            send_vw.append(float(send_vel.group(6)))
            send_steer_angle.append(float(send_vel.group(7)))
        if get_vel:
            get_vel_t.append(float(get_vel.group(1))*60.0*60.0+float(get_vel.group(2))*60.0+float(get_vel.group(3)))
            get_vx.append(float(get_vel.group(4)))
            get_vw.append(float(get_vel.group(5)))
            get_steer_angle.append(float(get_vel.group(6)))

## <span id="No3">解析文件，并画图 </span>

In [13]:
filename= 'robokit_2018-02-07_23-14-33.542.log'
t, x, y, theta, confidence = [], [], [], [], []
yaw_t, yaw_data, yaw_time = [], [], []
ax, ay, az = [], [], []
gx, gy, gz = [], [], []
offx, offy, offz = [], [], []
odo_t, odo_time, odo_x, odo_y, odo_theta = [], [], [], [], []
vel_t, vx, vy, omega = [], [], [], []
send_vel_t, send_vx, send_vy, send_vw, send_steer_angle = [], [], [], [], []
get_vel_t, get_vx, get_vw, get_steer_angle = [], [], [], []
read_log(filename)

In [14]:
plt.figure(1)
plt.plot(t,confidence,'g--',label='confidence ')
plt.legend()

In [19]:
fig = plt.figure(6)
ax1 = fig.add_subplot(111)
ax1.plot(t,x,'b-',label='x')
ax1.plot(t,y,'r-',label='y')
ax1.legend(loc = 'upper right')
ax2 = ax1.twinx()
ax2.plot(vel_t,vx,'g.',label='vx')
ax2.plot(vel_t,omega,'y.',label='omega')
ax2.legend(loc = 'upper left')